# Statistical Testing in behapy

This notebook demonstrates how to use the statistical testing module in `behapy` to compare behavioral metrics between groups, analyze transition matrices, and visualize effect sizes.

In [ ]:
import behapy as bp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)

## 1. Create Mock Data

We'll create two groups (Control and Treatment) with different behavioral profiles.

In [ ]:
def create_mock_bdata(group_type='control'):
    n_frames = 2000
    X = np.random.rand(n_frames, 5)
    
    if group_type == 'control':
        # Control: balanced behaviors, lower speed
        behaviors = np.random.choice(['Grooming', 'Walking', 'Rearing'], n_frames, p=[0.4, 0.4, 0.2])
        speed = np.random.normal(5, 1, n_frames)
    else:
        # Treatment: more walking, higher speed
        behaviors = np.random.choice(['Grooming', 'Walking', 'Rearing'], n_frames, p=[0.2, 0.6, 0.2])
        speed = np.random.normal(8, 1.5, n_frames)
        
    obs = pd.DataFrame({
        'leiden': behaviors,
        'speed': speed
    })
    
    bdata = bp.BehapyData(X=X, obs=obs)
    
    # Precompute metrics
    bp.tl.compute_transitions(bdata)
    bp.tl.detect_bouts(bdata)
    bp.tl.compute_bout_statistics(bdata)
    
    return bdata

ctrl_bdatas = [create_mock_bdata('control') for _ in range(5)]
treat_bdatas = [create_mock_bdata('treatment') for _ in range(5)]

all_bdatas = ctrl_bdatas + treat_bdatas
groups = ['Control']*5 + ['Treatment']*5

## 2. Group Comparisons

Compare speed between groups using a permutation test.

In [ ]:
results = bp.tl.compare_groups(
    all_bdatas, 
    groups, 
    metric='speed', 
    test='permutation', 
    n_permutations=1000
)
print("Speed Comparison Results:")
display(results)

## 3. Transition Matrix Analysis

Compare transition probabilities between a control and treatment subject.

In [ ]:
trans_results = bp.tl.test_transition_matrix(ctrl_bdatas[0], treat_bdatas[0], test='permutation', n_permutations=500)
print("Significant Transitions (p < 0.05):")
display(trans_results[trans_results['significant']].head())

## 4. Visualization

Visualize effect sizes and statistical summaries.

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
bp.pl.effect_sizes(results, plot_type='bar', title='Effect Size: Speed')

plt.subplot(1, 2, 2)
bp.pl.transition_matrix(ctrl_bdatas[0], comparison_bdata=treat_bdatas[0], show_significance=True)

plt.tight_layout()
plt.show()

## 5. Statistical Summary Figure

Create a multi-panel summary of all behavioral differences.

In [ ]:
fig = bp.pl.statistical_summary(
    all_bdatas, 
    groups, 
    metrics=['speed', 'bout_duration', 'bout_count'],
    figsize=(10, 12)
)
plt.show()